In [14]:
# import os, sys
# from google.colab import drive


# drive.mount('/content/drive', force_remount=True)
# package_path = '/content/drive/My\ Drive/Capstone/packages'
# sys.path.insert(0, package_path)

Mounted at /content/drive


In [15]:
# ! $package_path

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
/bin/bash: line 1: /content/drive/My Drive/Capstone/packages: Is a directory


In [7]:
# !pip install --target=$package_path langchain langchain_community gradio beautifulsoup4 unstructured jq kaleido sentence-transformers llama-cpp-python chromadb==0.4.14 rank_bm25 ragas

  Using cached langchain-0.1.12-py3-none-any.whl (809 kB)
  Using cached langchain_community-0.0.28-py3-none-any.whl (1.8 MB)
  Using cached gradio-4.21.0-py3-none-any.whl (17.0 MB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached unstructured-0.12.6-py3-none-any.whl (1.8 MB)
  Using cached jq-1.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (656 kB)
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)
  Using cached sentence_transformers-2.5.1-py3-none-any.whl (156 kB)
  Using cached llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_35_x86_64.whl
  Using cached chromadb-0.4.14-py3-none-any.whl (448 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
  Using cached ragas-0.1.4-py3-none-any.whl (73 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached pydantic-2.6.4-py3-none-any.whl (394 kB)
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)


In [2]:
from langchain_community.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader, TextLoader, JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import LlamaCpp
import pickle

In [11]:
# os.chdir('/content/drive/MyDrive/Capstone')
# !ls .

'02-19 - Literature Review Mini-Deliverable.gdoc'
'02-26 - Capstone Project Proposal.gdoc'
'03-04 - Software Tools Mini-Deliverable.gdoc'
'03-11 - Data Leakage Plan Mini-Deliverable.gdoc'
'03-11 - First Slack Stand-up Reports and Responses.gdoc'
'03-17 - Mentor Check-In.gdoc'
'03-18 - Report Outline Mini-Deliverable.gdoc'
'03-25 - Second Slack Stand-up Reports and Responses.gdoc'
'03-25 - Visuals Mini-Deliverable.gdoc'
'04-01 - Revised Outline Mini-Deliverable???.gdoc'
'04-08 - GitHub Repository Mini-Deliverable.gdoc'
'04-15 - Capstone Project Pre-Flight Checklist.gdoc'
'04-16 - Third Slack Stand-up Reports and Responses.gdoc'
'04-22 - Final Report.gdoc'
'Capstone Project Specifications Checklist.gdoc'
 chromadb
 evaluation_set_llm.ipynb
'LLM Inference Test.ipynb'
 models
 original_syllabi
 packages
'Project Tracker.gdoc'
 syllabus_loader_embeddings
 syllabus_loader_embeddings_ORIGINAL
 SyllabusLoader.ipynb
 transcripts
 updated_syllabi


In [3]:
persist_directory = "./syllabus_loader_embeddings/"

with open(f"{persist_directory}embeddings.pickle", 'rb') as handle:
    embeddings = pickle.load(handle)


/Users/Pat/Documents/umich/2024-02 SIADS 699/capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vectordb = Chroma(
    "syllabi",
    embedding_function=embeddings,
    persist_directory=persist_directory,
    collection_metadata={"hnsw:space": "cosine"},
)

In [5]:
retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20},
)

In [6]:
llm_open = LlamaCpp(
    model_path="./llama-2-7b.Q4_K_M.gguf",
    n_ctx=4096,  # 4096 for Llama, 32*1024 for Mistral
    n_gpu_layers=50,
    temperature=0.15,
    top_p=1,
    top_k=40,
    repeat_penalty=1.1,
    max_tokens=1024,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    stream=True,
)

/Users/Pat/Documents/umich/2024-02 SIADS 699/capstone/.venv/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_

In [7]:
def process_llm_response(llm_response):
    print("\n\nSources:")
    for source in llm_response["source_documents"]:
        print(source.metadata["source"])

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_open,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

In [9]:
query = "Tell me what I need to turn in for the capstone project."
llm_response = qa_chain(query)
process_llm_response(llm_response)

/Users/Pat/Documents/umich/2024-02 SIADS 699/capstone/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


ggml_metal_graph_compute_block_invoke: error: unsupported op 'RMS_NORM'
ggml_metal_graph_compute_block_invoke: error: unsupported op 'RMS_NORM'
ggml_metal_graph_compute_block_invoke: error: unsupported op 'MUL_MAT'
ggml_metal_graph_compute_block_invoke: error: unsupported op 'MUL_MAT'


: 

In [ ]:
llm_response